# Define class StockCracker

In [13]:
import numpy as np
np.set_printoptions(threshold=np.nan)
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
#---- matplotlib, my old favorite...
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='ticks')
#---- for Bokeh, a ploter
import bokeh.plotting as bp
from bokeh.layouts import gridplot, column
from bokeh.models import LinearAxis, DataRange1d, HoverTool, BoxSelectTool
TOOLS = ["pan,wheel_zoom,box_zoom,reset,crosshair, hover, box_select"]
WIDTH = 850
HEIGHT = 350
bp.output_notebook()
#---- for color, style print in iPython
from IPython.display import HTML, display
import datetime
import os
#---- for colorful print
from colorama import init
init(autoreset=True)
from colorama import Fore, Back, Style
#---- for statistics
from scipy import stats

Loading BokehJS ...

In [14]:
from updateData import Tickers
from StockCracker import StockCracker
from StockCracker import LabelManager

sc = StockCracker()
label = LabelManager()
data_path = Tickers()

Initializing StockCracker instance


In [15]:
#---------------
# read data
#---------------
data_index = sc.readOneCsv_Yahoo(data_path.tickers['^N225'] + 'data.csv')

data_stock = sc.readOneCsv_Yahoo(data_path.tickers['7203.T'] + 'data.csv')


In [16]:
# plot data
fig1 = bp.figure(x_axis_type="datetime", tools=TOOLS, plot_width=WIDTH, plot_height=HEIGHT, toolbar_location="above", title="Nikkei & Stock")
## Panasonic
fig1.extra_y_ranges = {"Stock":DataRange1d(start=data_stock[label.OPEN].min()-100, 
                                           end=data_stock[label.OPEN].max()+100)}
fig1.add_layout(LinearAxis(y_range_name = "Stock"), 'right')
fig1.line(data_stock[label.DATE], data_stock[label.OPEN], line_width=1, color="blue", y_range_name="Stock", legend="Stock")
## Nikkei
fig1.line(data_index[label.DATE], data_index[label.OPEN], line_width=1, color="red", legend="Nikkei")
fig1.legend.location="top_left"
fig1.legend.click_policy="hide"

bp.show(fig1)

In [17]:
#---------------
# 计算Beta系数
#---------------
# calculate Beta Coefficient, the input data must have the same length
beta, dailyChange_stock, dailyChange_index = sc.calcBetaCoefficient(data_stock, data_index)
##--- another version of beta, refer to ([ref] Beta Coefficient.pdf )
slope, intercept, r_value, p_value, std_err = stats.linregress(dailyChange_index, dailyChange_stock)
print("y = {}*x + {}".format(slope, intercept)) #line = slope*xi+intercept

Beta = 0.914382106359726
Target Stock is LESS volitale than the index, target stock is NOT risky
y = 0.914382106359726*x + 0.00011076795023501717


In [18]:
#---- calculate several Key Indicators
#----- OBV
#--------------
# 计算OBV，配合成交Volume
#--------------
# data_Mazda[['出来高','終値']] # extract 2 specific columns from original dataframe
OBV_stock = sc.keyIndicator_OBV(data_stock)
fig_OBV = bp.figure( x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="On-balance volume & Volume Bar of Mazda")
### plot volume
index_red = data_stock.loc[data_stock[label.CLOSE] >= data_stock[label.OPEN]].index  # days price up
index_blk = data_stock.loc[data_stock[label.CLOSE] <  data_stock[label.OPEN]].index  # days price down
fig_OBV.vbar(x=data_stock[label.DATE][index_red], bottom=0,
           top=data_stock[label.VOLUME][index_red], color="red", width=0.0, alpha=0.3, line_width=3.0, legend='up')
fig_OBV.vbar(x=data_stock[label.DATE][index_blk], bottom=0,
           top=data_stock[label.VOLUME][index_blk], color="green", width=0.0, alpha=0.3, line_width=3.0, legend='down')
### plot OBV
scaler = 0.1 # the trajectory/trend of OBV is crucial, the value is not
fig_OBV.line(data_stock[label.DATE], OBV_stock * scaler, color="black", alpha=1.0, legend='stockOBV')
### plot price 始値
fig_OBV.extra_y_ranges = {"Stock":DataRange1d(start=data_stock[label.OPEN].min()-6000, end=data_stock[label.OPEN].max()+100)}
fig_OBV.add_layout(LinearAxis(y_range_name = "Stock"), 'right')
fig_OBV.line(data_stock[label.DATE], data_stock[label.OPEN], line_width=2, color="blue", y_range_name="Stock", alpha=0.3, legend='stockOPEN')
fig_OBV.legend.location="top_left"
fig_OBV.legend.click_policy="hide"

bp.show(fig_OBV)

In [19]:
#----------------------------------
# 线性回归，用来用OBV计算走势
#----------------------------------
# Sliding window to calculate Trend of OBV
windowSize = 100
slope, intercept, r_value, p_value, std_err = stats.linregress( range(0, windowSize), data_stock[label.OPEN][0:windowSize] )
print(slope)
slope, intercept, r_value, p_value, std_err = stats.linregress( range(0, windowSize), OBV_stock[0:windowSize] )
print(slope)


9.98751875187519
460508.22682268225


In [20]:
# try MACD
## plot SMA100, SMA10 and price, trying to check the difference
#----- Simple moving average
SMA100_stock = sc.calc_SMA(data_stock, 100)
SMA10_stock = sc.calc_SMA(data_stock, 10)

fig_SMA = bp.figure( x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="SMA of stock")
fig_SMA.scatter(data_stock[label.DATE], SMA100_stock, color="red", alpha=0.6, legend="SMA 100")
fig_SMA.scatter(data_stock[label.DATE], SMA10_stock, alpha=0.6, legend="SMA 10")
### plot price 始値
fig_SMA.line(data_stock[label.DATE], data_stock[label.CLOSE], line_width=2, color="purple", alpha=0.3, legend="Price")
fig_SMA.legend.click_policy="hide"

bp.show(fig_SMA)

In [21]:
#----- Exponential moving average
# idea, the crossover of EMA100 and SMA100 also seems to be useful~
EMA100_stock = sc.calc_EMA(data_stock, 100)
EMA10_stock = sc.calc_EMA(data_stock, 10)

fig_EMA = bp.figure(x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="EMA of stock")
fig_EMA.scatter(data_stock[label.DATE], EMA100_stock, color="red", alpha=0.6, legend="EMA 100")
fig_EMA.scatter(data_stock[label.DATE], SMA100_stock, color="black", alpha=0.6, legend="SMA 100")
fig_EMA.scatter(data_stock[label.DATE], EMA10_stock, color="blue", alpha=0.6, legend="EMA 10")
### plot price 始値
fig_EMA.line(data_stock[label.DATE], data_stock[label.CLOSE], line_width=2, color="purple", alpha=0.3, legend="Price")
fig_EMA.legend.click_policy="hide"
bp.show(fig_EMA)

In [22]:
#-------------------------#
# plot conventional MACD(12, 26, 9)
#-------------------------#
MACD, signal = sc.keyIndicator_MACD(data_stock, 12, 26, 9)

fig_MACD = bp.figure(x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="MACD of stock")
fig_MACD.line(data_stock[label.DATE], MACD, color='red', legend="MACD")
fig_MACD.line(data_stock[label.DATE], signal, legend="signal")
#fig_MACD.line(data_Mazda['日付'], MACD - signal, legend="MACD-Signal")
### plot price 終値
fig_MACD.line(data_stock[label.DATE], data_stock[label.CLOSE]/10, line_width=2, color="purple", alpha=0.3)
fig_MACD.legend.click_policy="hide"
bp.show(fig_MACD)

In [23]:
#----------------------
# 看MACD-Signal，
#  >0:红色 
#  <0:绿色
#  Cyan cross: 红绿变化点～
#----------------------
MACD, signal = sc.keyIndicator_MACD(data_stock, 12, 26, 9)

data = MACD - signal
index_up    = []
index_down  = []
index_cross = []
for idx in range(len(data)):
    if data[idx] > 0:
        index_up.append(idx)
    else:
        index_down.append(idx)
for idx in range(len(data) - 1):
    if data[idx] < 0 and data[idx+1] > 0:
        index_cross.append(idx)
        
#------- linear Regression
slope_cross = []
#for ele in index_cross:
for idx in range(len(data_stock)):
    oneSlice = sc.focusOn(data_stock, idx, 2, 2)
    slope, intercept, r_value, p_value, std_err = stats.linregress([i for i in range(len(oneSlice))], oneSlice[label.CLOSE])
    slope_cross.append(slope)
    
#------- Plot
fig_MACD = bp.figure(x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="MACD of stock")
### plot price 終値
fig_MACD.line(data_stock[label.DATE], data_stock[label.CLOSE], line_width=2, color="purple", alpha=0.3, legend="Price")
fig_MACD.scatter(data_stock[label.DATE][index_up], data_stock[label.CLOSE][index_up], 
                 line_width=0.1, color="red", alpha=0.8, legend="Buy?")
fig_MACD.scatter(data_stock[label.DATE][index_down], data_stock[label.CLOSE][index_down], 
                 line_width=0.1, color="green", alpha=0.8, legend="Sell?")
### MACD - Signal
fig_MACD.extra_y_ranges = {"MACD of stock":DataRange1d(start=-100, end=100)}
fig_MACD.add_layout(LinearAxis(y_range_name = "MACD of stock"), 'right')
fig_MACD.line(data_stock[label.DATE], MACD - signal, legend="MACD-Signal", y_range_name="MACD of stock", alpha=0.3)
### crossover
fig_MACD.circle_cross(data_stock[label.DATE][index_cross], data_stock[label.CLOSE][index_cross], line_width=1, fill_alpha=0.2,
                      color="cyan", size=10, legend="cross(buy)")
### slope
fig_MACD.vbar(x=data_stock[label.DATE], bottom=0,
           top=slope_cross, color="red", width=0.0, alpha=0.3, line_width=3.0)
fig_MACD.legend.click_policy="hide"

bp.show(fig_MACD)

In [24]:
"""RSI
"""
RSI = sc.keyIndicator_RSI(data_stock, 14)

fig_RSI = bp.figure(x_axis_type = "datetime", tools=TOOLS, 
                    plot_width=WIDTH, plot_height=HEIGHT + 300, 
                    toolbar_location="above", title="RSI of stock")
fig_RSI.line(data_stock[label.DATE], RSI, color='red', legend="RSI")
### plot price 終値
fig_RSI.line(data_stock[label.DATE], data_stock[label.CLOSE]/10, line_width=2, color="purple", alpha=0.3)
fig_RSI.legend.click_policy="hide"
bp.show(fig_RSI)